In [ ]:
# !pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("LinearRegressionExample").getOrCreate()

In [ ]:
df = spark.read.csv('../../data/Advertising.csv', header = True, inferSchema = True)
df.printSchema()

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Step 3: Prepare the features and label
# Combine the feature columns ('TV', 'Radio', 'Newspaper') into a single feature vector
assembler = VectorAssembler(inputCols=["TV", "Radio", "Newspaper"], outputCol="features")
df = assembler.transform(df)

# Select the relevant columns for the regression model
df = df.select("features", "Sales")

In [ ]:
df.toPandas()

In [ ]:
train_data, test_data = df.randomSplit([0.8, 0.2])

In [ ]:
from pyspark.ml.regression import LinearRegression

# Initialize the Linear Regression model
lr = LinearRegression(featuresCol="features", labelCol="Sales")

# Fit the model on the training data
lr_model = lr.fit(train_data)

In [ ]:
# Step 6: Make Predictions
predictions = lr_model.transform(test_data)
predictions.select("features", "Sales", "prediction").show()

In [ ]:
training_summary = lr_model.summary

# Print the coefficients and intercept for linear regression
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

# Show the RMSE and R-squared
print("RMSE: %f" % training_summary.rootMeanSquaredError)
print("R-squared: %f" % training_summary.r2)


# Make Prediction

In [ ]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# Create a DataFrame for the new data
new_data = [Row(features=Vectors.dense(150.0, 25.0, 20.0))]
new_df = spark.createDataFrame(new_data)


In [ ]:
# Make a prediction using the new data
new_predictions = lr_model.transform(new_df)

# Show the prediction
new_predictions.select("features", "prediction").toPandas()